In [52]:
import sys
import hashlib
from multiprocessing import Pool, cpu_count
import numpy as np

def get_key(passwd):
    # Create 256-byte key by repeating the password
    key = bytearray(256)
    j = 0
    for i in range(256):
        key[i] = ord(passwd[j]) if j < len(passwd) else 0
        if j < len(passwd):
            j += 1
        else:
            j = 0
    return key

class RC4Generator:
    def __init__(self, key):
        self.s = list(range(256))
        self.i = 0
        self.j = 0
        
        # Key scheduling algorithm
        j = 0
        for i in range(256):
            j = (j + self.s[i] + key[i]) % 256
            self.s[i], self.s[j] = self.s[j], self.s[i]

    def rand(self):
        self.i = (self.i + 1) % 256
        self.j = (self.j + self.s[self.i]) % 256
        self.s[self.i], self.s[self.j] = self.s[self.j], self.s[self.i]
        
        t = (self.s[self.i] + self.s[self.j]) % 256
        return self.s[t]

def encrypt_decrypt(passwd, input_data):
    key = get_key(passwd)
    generator = RC4Generator(key)
    
    return bytearray(b ^ generator.rand() for b in input_data)

def calculate_md5(file_path):
    with open(file_path, "rb") as file:
        md5_hash = hashlib.md5()
        while chunk := file.read(8192):
            md5_hash.update(chunk)
        return md5_hash.hexdigest()
    
def read_file(name):
    try:
        with open(name, 'rb') as file:
            return bytearray(file.read())
    except FileNotFoundError:
        print(f"File '{name}' not found.")
        sys.exit(1)

def write_file(name, data):
    with open(name, 'wb') as file:
        file.write(data)
    
def is_valid_decryption(decrypted_data):
    # Vectorized ASCII count for better performance
    ascii_bytes = np.array(decrypted_data)
    ascii_count = np.count_nonzero((32 <= ascii_bytes) & (ascii_bytes <= 126))
    
    # Calculate percentage
    return (ascii_count / len(decrypted_data)) >= 0.75

def worker(task):
    encrypted_data, password = task
    decrypted_data = encrypt_decrypt(password, encrypted_data)

    if is_valid_decryption(decrypted_data):
        return password
    return None

def brute_force_decrypt(input_file_path):
    encrypted_data = read_file(input_file_path)
    
    passwords = (f"{i:06d}" for i in range(100000, 1000000))
    
    completed_tasks = 0
    valid_passwords = [] 
    
    with Pool(cpu_count()) as pool:
        tasks = ((encrypted_data, password) for password in passwords)
        for result in pool.imap_unordered(worker, tasks, chunksize=5000):
            completed_tasks += 1
            
            if result:
                print(f"Valid password found: {result}")
                valid_passwords.append(result)
    
    return valid_passwords

def decode_message(encrypted_data, password):
    return encrypt_decrypt(password, encrypted_data).decode('utf-8')

def main():
    encrypted_files = ['text1_enc.txt', 'text2_enc.txt', 'text3_enc.txt', 'text4_enc.txt']
    md5s = ['4622c7bcb17d81c081baec766fb6fdc2', '95048f6ded12b35a87015078c236abf2', 
            'be5401874206c8066a255ad39f127ca2', '8314a669612496867c3388c76e783bf7']
    
    for encrypted_file, expected_md5 in zip(encrypted_files, md5s):
        print(f'Starting file {encrypted_file}')
        if expected_md5 != calculate_md5(encrypted_file):
            raise Exception(f'MD5 Error for file: {encrypted_file}')
        
        valid_passwords = brute_force_decrypt(encrypted_file)
        
        for password in valid_passwords:
            decrypted_message = decode_message(read_file(encrypted_file), password)
            if decrypted_message:
                print(f"Decrypted message with password {password}: {decrypted_message}")

if __name__ == '__main__':
    main()


Starting file text1_enc.txt
Valid password found: 123456
Decrypted message with password 123456: Orange začal svoje vysielače napájať solárne. Využíva novú technológiu od Huawei

Inovatívne riešenie so solárnymi panelmi chránenými proti krádeži a inteligentnými batériami na každej základňovej stanici ušetrí až štyristopäťdesiatpäť kg emisií kysličníka uhličitého ročne.

Orange sa ako prvý na Slovensku pustil s technologickou podporou čínskej spoločnosti Huawei do naozaj ambiciózneho projektu. Rozhodol sa dve tretiny svojich základňových staníc (BTS) vybaviť pomerne efektívnym solárnym napájaním s návratnosťou investície už do šesť rokov. Dokonca aj v rámci Európy patrí k prvým operátorom, ktorí prešli od testovania takéhoto napájania k jeho plošnému komerčnému nasadeniu prakticky na celú sieť, samozrejme tam, kde je to technicky možné.

Starting file text2_enc.txt
Valid password found: 555555
Decrypted message with password 555555: Robot vo fast-foode nahradil pri smažení hranolčekov ľ